In [1]:
import os, joblib
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Bidirectional, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import torch
import torch.distributed as dist 
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense
import numpy as np
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

2024-11-25 21:58:13.366266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732553893.388182   14723 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732553893.395029   14723 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 21:58:13.416866: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define Data directory
DATA_DIR = '../data/'

# list of poet names
# Removed the 5 lowest poets
poets = ["abbasinYousuf", "ajmalKhattak", "allamaAbdulHai", "ghaniKhan", "hamzaBaba", "javedAhmedzai", "karanKhan", "khatirAfridi", "khushalKhanKhattak", "matiullahTurab", "mumtazOrakazi", "munirJan", "naeemAhmed", "rabiaMumtaz", "rahmanBaba", "rehmatShah", "sahibShahSabir", "shabbirKhanDurrani", "shakirOrakzai", "shoaibKhanKhattak"]

poet_labels = {poet: i for i, poet in enumerate(poets)}
print(poet_labels)

{'abbasinYousuf': 0, 'ajmalKhattak': 1, 'allamaAbdulHai': 2, 'ghaniKhan': 3, 'hamzaBaba': 4, 'javedAhmedzai': 5, 'karanKhan': 6, 'khatirAfridi': 7, 'khushalKhanKhattak': 8, 'matiullahTurab': 9, 'mumtazOrakazi': 10, 'munirJan': 11, 'naeemAhmed': 12, 'rabiaMumtaz': 13, 'rahmanBaba': 14, 'rehmatShah': 15, 'sahibShahSabir': 16, 'shabbirKhanDurrani': 17, 'shakirOrakzai': 18, 'shoaibKhanKhattak': 19}


In [3]:
# Load and Label the Data
def load_and_label(data_dir, poets, poet_labels):
    data = []
    labels = []

    for poet in poets:
        poet_dir = os.path.join(data_dir, poet)
        file_path = os.path.join(poet_dir, f'{poet}.txt')

        # Check if the file exists
        if not os.path.exists(file_path):
            print(f'{file_path} does not exist')
            continue

        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            data.extend(lines)
            labels.extend([poet_labels[poet]] * len(lines))

    df = pd.DataFrame({'text': data, 'label': labels})
    return data, labels, df

In [4]:
data, labels, df = load_and_label(DATA_DIR, poets, poet_labels)
df.describe()

,label
count,54620.000000
mean,8.625265
std,5.352617
min,0.000000
25%,4.000000
50%,8.000000
75%,14.000000
max,19.000000


In [6]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
device

'cpu'

In [7]:
poems_train, poems_temp, lables_train, labels_temp = train_test_split(data, labels, test_size=0.2, random_state=42)
poems_val, poems_test, labels_val, labels_test = train_test_split(poems_temp, labels_temp, test_size=0.5, random_state=42)

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")
model = AutoModelForMaskedLM.from_pretrained("urduhack/roberta-urdu-small")

model.to(device)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of the model checkpoint at urduhack/roberta-urdu-small were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

model.safetensors:   0%|          | 0.00/507M [00:00<?, ?B/s]

### XLM roBERTa Large

In [9]:
# Function to save embeddings
def save_embeddings(texts, prefix, model, tokenizer, device, max_length=80):
    for i, text in enumerate(texts):
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
            # Use the mean of the last hidden state as the embedding
            embedding = outputs.hidden_states[-1].mean(dim=1).to(device)
        
        torch.save(embedding, f'./models_roberta_urdu/{prefix}_embedding_{i}.pt')
        if i % 100 == 0:
            print(f"{prefix.capitalize()} Embedding {i} saved.")


In [10]:
# Save embeddings for train, test and validation
save_embeddings(poems_train, 'train', model, tokenizer, device)

Train Embedding 0 saved.
Train Embedding 100 saved.
Train Embedding 200 saved.
Train Embedding 300 saved.
Train Embedding 400 saved.
Train Embedding 500 saved.
Train Embedding 600 saved.
Train Embedding 700 saved.
Train Embedding 800 saved.
Train Embedding 900 saved.
Train Embedding 1000 saved.
Train Embedding 1100 saved.
Train Embedding 1200 saved.
Train Embedding 1300 saved.
Train Embedding 1400 saved.
Train Embedding 1500 saved.
Train Embedding 1600 saved.
Train Embedding 1700 saved.
Train Embedding 1800 saved.
Train Embedding 1900 saved.
Train Embedding 2000 saved.
Train Embedding 2100 saved.
Train Embedding 2200 saved.
Train Embedding 2300 saved.
Train Embedding 2400 saved.
Train Embedding 2500 saved.
Train Embedding 2600 saved.
Train Embedding 2700 saved.
Train Embedding 2800 saved.
Train Embedding 2900 saved.
Train Embedding 3000 saved.
Train Embedding 3100 saved.
Train Embedding 3200 saved.
Train Embedding 3300 saved.
Train Embedding 3400 saved.
Train Embedding 3500 saved.
Trai

In [11]:
save_embeddings(poems_val, 'val', model, tokenizer, device)

Val Embedding 0 saved.
Val Embedding 100 saved.
Val Embedding 200 saved.
Val Embedding 300 saved.
Val Embedding 400 saved.
Val Embedding 500 saved.
Val Embedding 600 saved.
Val Embedding 700 saved.
Val Embedding 800 saved.
Val Embedding 900 saved.
Val Embedding 1000 saved.
Val Embedding 1100 saved.
Val Embedding 1200 saved.
Val Embedding 1300 saved.
Val Embedding 1400 saved.
Val Embedding 1500 saved.
Val Embedding 1600 saved.
Val Embedding 1700 saved.
Val Embedding 1800 saved.
Val Embedding 1900 saved.
Val Embedding 2000 saved.
Val Embedding 2100 saved.
Val Embedding 2200 saved.
Val Embedding 2300 saved.
Val Embedding 2400 saved.
Val Embedding 2500 saved.
Val Embedding 2600 saved.
Val Embedding 2700 saved.
Val Embedding 2800 saved.
Val Embedding 2900 saved.
Val Embedding 3000 saved.
Val Embedding 3100 saved.
Val Embedding 3200 saved.
Val Embedding 3300 saved.
Val Embedding 3400 saved.
Val Embedding 3500 saved.
Val Embedding 3600 saved.
Val Embedding 3700 saved.
Val Embedding 3800 saved

In [12]:
save_embeddings(poems_test, 'test', model, tokenizer, device)

Test Embedding 0 saved.
Test Embedding 100 saved.
Test Embedding 200 saved.
Test Embedding 300 saved.
Test Embedding 400 saved.
Test Embedding 500 saved.
Test Embedding 600 saved.
Test Embedding 700 saved.
Test Embedding 800 saved.
Test Embedding 900 saved.
Test Embedding 1000 saved.
Test Embedding 1100 saved.
Test Embedding 1200 saved.
Test Embedding 1300 saved.
Test Embedding 1400 saved.
Test Embedding 1500 saved.
Test Embedding 1600 saved.
Test Embedding 1700 saved.
Test Embedding 1800 saved.
Test Embedding 1900 saved.
Test Embedding 2000 saved.
Test Embedding 2100 saved.
Test Embedding 2200 saved.
Test Embedding 2300 saved.
Test Embedding 2400 saved.
Test Embedding 2500 saved.
Test Embedding 2600 saved.
Test Embedding 2700 saved.
Test Embedding 2800 saved.
Test Embedding 2900 saved.
Test Embedding 3000 saved.
Test Embedding 3100 saved.
Test Embedding 3200 saved.
Test Embedding 3300 saved.
Test Embedding 3400 saved.
Test Embedding 3500 saved.
Test Embedding 3600 saved.
Test Embeddin

In [13]:
class_labels = sorted(list(set(lables_train)))

def load_embeddings(prefix, num_embeddings):
    embeddings = []
    for i in range(num_embeddings):
        embedding = torch.load(f'./models_roberta_urdu/{prefix}_embedding_{i}.pt')
        embeddings.append(embedding)
    return torch.stack(embeddings)

In [14]:
train_embeddings = load_embeddings('train', len(poems_train))
train_embeddings

/tmp/ipykernel_14723/1025018291.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embedding = torch.load(f'./models_roberta_urdu/{prefix}_embedding_{i}.pt')


tensor([[[ 0.7200, -0.0091,  0.1667,  ...,  0.3693,  1.6527, -0.0542]],

        [[ 0.8675,  0.2121,  0.2090,  ...,  0.0997,  1.1922, -0.3499]],

        [[ 0.6282, -0.2588, -0.1849,  ...,  0.5559,  1.3151, -0.2796]],

        ...,

        [[ 0.7985,  0.5469,  0.0372,  ...,  0.4290,  1.6407, -0.1550]],

        [[ 0.7071,  0.5777,  0.0671,  ...,  0.6072,  1.5448, -0.0921]],

        [[ 0.5011, -0.2120,  0.0715,  ...,  0.2596,  1.7465, -0.2179]]])

In [15]:
val_embeddings = load_embeddings('val', len(poems_val))
val_embeddings

/tmp/ipykernel_14723/1025018291.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embedding = torch.load(f'./models_roberta_urdu/{prefix}_embedding_{i}.pt')


tensor([[[ 0.8267, -0.2748,  0.4003,  ...,  0.3170,  1.0390, -0.0601]],

        [[ 0.7834, -0.0339, -0.0681,  ...,  0.4437,  1.3576, -0.0454]],

        [[ 0.8073,  0.4141,  0.1188,  ...,  0.6287,  1.4653, -0.2032]],

        ...,

        [[ 0.5644, -0.0054,  0.0542,  ...,  0.2336,  1.6979, -0.2195]],

        [[ 0.5286,  0.0336,  0.0806,  ...,  0.2093,  1.7248, -0.2653]],

        [[ 0.5498,  0.1596, -0.0471,  ...,  0.7972,  1.1110, -0.3658]]])

In [16]:
test_embeddings = load_embeddings('test', len(poems_test))
test_embeddings

/tmp/ipykernel_14723/1025018291.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embedding = torch.load(f'./models_roberta_urdu/{prefix}_embedding_{i}.pt')


tensor([[[ 0.5880, -0.1590,  0.0435,  ...,  0.0070,  1.6093, -0.3489]],

        [[ 0.5811, -0.1360, -0.0797,  ...,  0.6468,  1.3318, -0.2723]],

        [[ 0.8738, -0.1279, -0.0895,  ...,  0.5300,  1.4722, -0.3882]],

        ...,

        [[ 1.1735,  0.4458,  0.2705,  ...,  0.8301,  1.1468, -0.0167]],

        [[ 0.7532,  0.2574,  0.0437,  ...,  0.5354,  1.6378, -0.0621]],

        [[ 0.4559, -0.1600,  0.1205,  ...,  0.1971,  1.7472, -0.1388]]])

In [17]:
train_labels = torch.tensor(lables_train)
val_labels = torch.tensor(labels_val)
test_labels = torch.tensor(labels_test)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
# encoded_labels = label_encoder.fit_transform(lables_train)

In [18]:
train_embeddings_np = train_embeddings.cpu().detach().numpy()
val_embeddings_np = val_embeddings.cpu().detach().numpy()
test_embeddings_np = test_embeddings.cpu().detach().numpy()
labels_train_np = np.array(lables_train)
labels_val_np = np.array(labels_val)
labels_test_np = np.array(labels_test)

In [19]:
train_embeddings_np.shape, val_embeddings_np.shape, test_embeddings_np.shape, labels_train_np.shape, labels_val_np.shape, labels_test_np.shape

((43696, 1, 768), (5462, 1, 768), (5462, 1, 768), (43696,), (5462,), (5462,))

In [20]:
#BERT WITH BERT

class PoemClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(PoemClassifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten along the sequence length dimension
        x = F.relu(x)
        x = self.fc(x)
        return x

In [21]:
# set the input size based on the size of the embeddings
input_size = train_embeddings.size(2)

# initialize the classifier and move it to the device
classifier = PoemClassifier(input_size, len(labels)).to(device)
# classifier = PoemClassifier(input_size, len(class_labels)).to(device)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    classifier = nn.DataParallel(classifier)

# Define the optimizer and loss function
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Create dataloader for training to iterate over the embeddings and labels in batches
train_dataset = TensorDataset(train_embeddings, train_labels)
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [22]:
val_accuracies = []
train_accuracies = []
best_val_loss = float('inf')
patience = 5  # You can adjust this based on your needs
counter = 0
train_losses = []  # Track training loss
val_losses = []

In [23]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(TensorDataset(val_embeddings, val_labels), batch_size = 8)
accumalation_steps = 4

In [24]:
train_loader, val_loader

(<torch.utils.data.dataloader.DataLoader at 0x7244301ff6e0>,
 <torch.utils.data.dataloader.DataLoader at 0x72440bff0890>)

In [ ]:
# Train Loop with early stopping
num_epochs = 20
for epoch in range(num_epochs):
    classifier.train()
    epoch_loss = 0
    optimizer.zero_grad()

    for i, (batch_embeddings, batch_labels) in enumerate(train_loader):
        if i % 100 == 0: print(f"Iteration {i}/{len(train_loader)} of train loader")
        batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)

        # optimizer.zero_grad()
        output = classifier(batch_embeddings)
        loss = criterion(output, batch_labels)
        # loss.backward()
        loss = loss / accumalation_steps
        loss.backward()
        # optimizer.step()

        if (i + 1) % accumalation_steps == 0 or (i + 1) == len(train_loader):
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += loss.item() * accumalation_steps

    avg_epoch_loss = epoch_loss / len(train_loader.dataset)
    train_losses.append(avg_epoch_loss)

    print(f"Epoch {epoch + 1}/{num_epochs} - Training and Validation Accuracy:")

    # Training accuracy and data in batches to avoid over memory consumption
    classifier.eval()
    train_correct = 0
    train_total = 0
    with torch.no_grad():
        for batch_embeddings, batch_labels in train_loader:
            batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)
            train_outputs = classifier(batch_embeddings)
            train_probabilities = F.softmax(train_outputs, dim=1)
            train_predicted_labels = torch.argmax(train_probabilities, dim=1)
            train_correct += (train_predicted_labels == batch_labels).sum().item()
            train_total += batch_labels.size(0)
    train_accuracy = train_correct / train_total
    train_accuracies.append(train_accuracy)


    # Validation accuracy and loss in batches to avoid over memory consumption
    val_correct = 0
    val_total = 0
    val_loss_total = 0
    with torch.no_grad():
        for val_batch_embeddings, val_batch_labels in DataLoader(TensorDataset(val_embeddings, val_labels), batch_size=8):
            val_batch_embeddings, val_batch_labels = val_batch_embeddings.to(device), val_batch_labels.to(device)
            val_outputs = classifier(val_batch_embeddings)
            val_loss = criterion(val_outputs, val_batch_labels)
            val_loss_total += val_loss.item()

            val_probabilities = F.softmax(val_outputs, dim=1)
            val_predicted_labels = torch.argmax(val_probabilities, dim=1)
            val_correct += (val_predicted_labels == val_batch_labels).sum().item()
            val_total += val_batch_labels.size(0)

        val_accuracy = val_correct / val_total
        val_accuracies.append(val_accuracy)
        val_losses.append(val_loss_total / len(val_loader))
        

    # Print current status
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {avg_epoch_loss:.4f}, Val Loss: {val_loss_total / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Early stopping check
    if val_loss_total < best_val_loss:
        best_val_loss = val_loss_total
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early Stopping!")
            break

In [ ]:
# evaluation on the test set
with torch.no_grad():
    classifier.eval()
    test_outputs = classifier(test_embeddings.to(device))
    test_loss = criterion(test_outputs, test_labels.to(device))

    # convert logits to probs
    test_probabilities = F.softmax(test_outputs, dim=1)

    # get the predicted labels
    test_predicted_labels = torch.argmax(test_probabilities, dim=1).tolist()

    cm = confusion_matrix(test_labels, test_predicted_labels)

    # calculate the accuracy
    test_accuracy = accuracy_score(test_labels, test_predicted_labels)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    test_loss_value = test_loss.item()

In [ ]:
# Plot the confusion matrix
cm = confusion_matrix(labels_test_np, test_predicted_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


# Plot accuracy and loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(1, 3, 3)
plt.bar(["Test Loss"], [test_loss_value], color="skyblue")
plt.title("Test Loss")
plt.ylabel("Loss")

plt.tight_layout()
plt.show()